In [ ]:
!pwd

/content


In [ ]:
!nvidia-smi

Wed Dec 10 14:30:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# !pip install REMI-z

In [ ]:
# import os

# # Check if REMI-z directory exists before cloning
# if os.path.exists('REMI-z'):
#     print("REMI-z directory already exists. Skipping git clone.")
# else:
#     !git clone https://github.com/Sonata165/REMI-z.git

# # Change directory using %cd for persistent changes in Colab
# %cd REMI-z
# !pip install -r Requirements.txt
# !pip install -e .
# %cd ..

In [ ]:
from remi_z import MultiTrack

In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

## Dataset Preparation

In [ ]:
!wget http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz
!tar -xzf lmd_full.tar.gz

--2025-12-10 14:31:13--  http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz
Resolving hog.ee.columbia.edu (hog.ee.columbia.edu)... 128.59.66.5
Connecting to hog.ee.columbia.edu (hog.ee.columbia.edu)|128.59.66.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1768163879 (1.6G) [application/x-gzip]
Saving to: ‘lmd_full.tar.gz.1’

lmd_full.tar.gz.1   100%[===================>]   1.65G  7.61MB/s    in 3m 11s  

2025-12-10 14:34:25 (8.81 MB/s) - ‘lmd_full.tar.gz.1’ saved [1768163879/1768163879]



In [ ]:
!ls

lmd_full  lmd_full.tar.gz  lmd_full.tar.gz.1  REMI-z  sample_data


In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("amaai-lab/MidiCaps")
ds

DatasetDict({
    train: Dataset({
        features: ['location', 'caption', 'genre', 'genre_prob', 'mood', 'mood_prob', 'key', 'time_signature', 'tempo', 'tempo_word', 'duration', 'duration_word', 'chord_summary', 'chord_summary_occurence', 'instrument_summary', 'instrument_numbers_sorted', 'all_chords', 'all_chords_timestamps', 'test_set'],
        num_rows: 168385
    })
})

In [ ]:
df = ds['train'].to_pandas()
df.head()

,location,caption,genre,genre_prob,mood,mood_prob,key,time_signature,tempo,tempo_word,duration,duration_word,chord_summary,chord_summary_occurence,instrument_summary,instrument_numbers_sorted,all_chords,all_chords_timestamps,test_set
0,lmd_full/1/1a0751ad20e2f82957410a7510a1b13e.mid,A melodic electronic composition with classica...,"[electronic, classical]","[0.3596, 0.2367]","[melodic, film, space, epic, relaxing]","[0.1228, 0.1114, 0.0917, 0.0828, 0.079]",F# minor,4/4,135.0,Allegro,59,Short song,"[F#m, A/E, C#]",2,"[String Ensemble, Trumpet, Brass Section, Synt...","[48, 56, 61, 50, 128, 35]","[F#m, A/E, C#, F#m, C#, F#m, C#, D, C#, Bm, F#...","[0.464399092, 5.015510204, 5.758548752, 8.8235...",False
1,lmd_full/1/164fcc530a285bdb8066c6aa8616c95b.mid,A cheerful and melodic Christmas song that ble...,"[classical, pop]","[0.1914, 0.1836]","[christmas, happy, melodic, corporate, adventure]","[0.1763, 0.1688, 0.1643, 0.1282, 0.072]",Eb major,2/4,124.0,Allegro,169,Song,"[Eb, Bb7, Eb, Fm]",5,"[Tango Accordion, Clarinet, Acoustic Guitar, D...","[23, 71, 25, 128, 32]","[Bb7, Eb, Bb7, Eb, Fm, Eb, Bb7, Eb, Bb7, Eb, F...","[0.464399092, 6.780226757, 9.84526077, 13.7462...",False
2,lmd_full/1/16b9a230fb007c0009feee532c3c4686.mid,This energetic electronic and classical compos...,"[electronic, classical]","[0.2148, 0.1806]","[energetic, film, melodic, happy, dark]","[0.1079, 0.1062, 0.0958, 0.0927, 0.0635]",C major,4/4,120.0,Moderate tempo,158,Song,"[C, G, C, C7]",2,[],[],"[C, G, C, C7, Dm, C, G, C, C7, Bbmaj7, C, G7, ...","[0.464399092, 4.086712018, 7.430385487, 8.9164...",False
3,lmd_full/1/17655598958db48a34cd882f81402568.mid,A short electronic ambient song featuring a pi...,"[electronic, ambient]","[0.3031, 0.2369]","[film, energetic, melodic, epic, dark]","[0.1162, 0.1108, 0.105, 0.0863, 0.0824]",E major,4/4,170.0,Presto,99,Short song,"[B, Emaj7, C#m7, B7, A]",5,[Piano],[0],"[B, Emaj7, C#m7, B7, A, B, Emaj7, C#m7, B7, A,...","[0.464399092, 1.30031746, 2.972154195, 3.71519...",False
4,lmd_full/1/13ad61f0cef919bc069175d0b0ce2437.mid,"In this short electronic song, synth strings l...","[electronic, ambient]","[0.4655, 0.2017]","[film, space, relaxing, melodic, dark]","[0.0944, 0.0943, 0.0885, 0.0856, 0.0816]",C minor,4/4,124.0,Fast,103,Short song,"[Gm, Dm, F, C, Bb6]",1,"[Synth Strings, Drums, Electric Bass, Trumpet,...","[51, 128, 34, 56, 52, 115, 61, 25, 55, 16]","[Gm, Dm, F, C, Bb6, C7, Bb, Cm, Bb, Cm7, Bb, G...","[0.464399092, 4.272471655, 5.665668934, 8.2663...",False


In [ ]:
import numpy as np
np.random.seed(42)

stratified_sample_rows = []
sampled_indices = set()
TARGET_SAMPLE_SIZE = 500

# Explode the 'genre' column to get all unique genres
exploded_genres = df['genre'].explode()
list_of_unique_genres = exploded_genres.unique().tolist()

for genre in list_of_unique_genres:
    # Identify all rows that contain this specific genre and whose indices are not already in sampled_indices
    available_for_genre_df = df[df['genre'].apply(lambda x: genre in x)]
    available_for_genre_indices = available_for_genre_df.index.difference(sampled_indices)

    # Randomly select up to 100 indices from available_for_genre
    num_to_select = min(100, len(available_for_genre_indices))
    if num_to_select > 0:
        selected_indices = np.random.choice(available_for_genre_indices, num_to_select, replace=False)
        stratified_sample_rows.extend(selected_indices)
        sampled_indices.update(selected_indices)

print(f"Initial sample size after stratified sampling: {len(sampled_indices)}")

# If the total sample size is less than target sample size, randomly select additional unique rows
if len(sampled_indices) < TARGET_SAMPLE_SIZE:
    remaining_needed = TARGET_SAMPLE_SIZE - len(sampled_indices)

    # Identify all indices in df that are not yet in sampled_indices
    additional_available_indices = df.index.difference(sampled_indices)

    # Randomly select remaining_needed unique indices from additional_available_indices
    if len(additional_available_indices) >= remaining_needed:
        additional_selection = np.random.choice(additional_available_indices, remaining_needed, replace=False)
        stratified_sample_rows.extend(additional_selection)
        sampled_indices.update(additional_selection)
    else:
        # If not enough unique rows to reach target size, take all available and print a warning
        print(f"Warning: Could not reach {TARGET_SAMPLE_SIZE} samples. Only {len(sampled_indices) + len(additional_available_indices)} unique rows available.")
        stratified_sample_rows.extend(additional_available_indices)
        sampled_indices.update(additional_available_indices)

# Create the sample_df DataFrame using df.loc[] with the unique indices collected in sampled_indices
sample_df_stratified = df.loc[list(sampled_indices)]

print(f"Final stratified sample size: {len(sample_df_stratified)}")
print("Stratified Sample DataFrame head:")
print(sample_df_stratified.head())
print("Stratified Sample DataFrame shape:")
print(sample_df_stratified.shape)

Initial sample size after stratified sampling: 2842
Final stratified sample size: 2842
Stratified Sample DataFrame head:
                                               location  \
65537   lmd_full/2/2a604c93ed4ee9d82d388da694e89e7c.mid   
32776   lmd_full/0/0fd51dae1d5ffc871de6b8bd8ff3e5ce.mid   
40971   lmd_full/0/00bb4be2822c3828935498af7b90b98e.mid   
139280  lmd_full/e/e7394f75f6978369eb463abcb75a88da.mid   
49169   lmd_full/f/f5ac97af893d6b5cec0324bc49ae10bb.mid   

                                                  caption  \
65537   A cheerful reggae-pop song in C# major, featur...   
32776   A cheerful and inspiring pop song featuring ac...   
40971   A soothing Christmas pop song with jazz influe...   
139280  A lively electronic and pop fusion, this Chris...   
49169   A very short fragment of electronic music with...   

                             genre        genre_prob  \
65537                [reggae, pop]  [0.3014, 0.2529]   
32776               [pop, popfolk]  [0.5125, 

In [ ]:
df = sample_df_stratified[['location', 'genre']]
df.reset_index(drop=True, inplace=True)
display(df.head())

,location,genre
0,lmd_full/2/2a604c93ed4ee9d82d388da694e89e7c.mid,"[reggae, pop]"
1,lmd_full/0/0fd51dae1d5ffc871de6b8bd8ff3e5ce.mid,"[pop, popfolk]"
2,lmd_full/0/00bb4be2822c3828935498af7b90b98e.mid,"[pop, jazz]"
3,lmd_full/e/e7394f75f6978369eb463abcb75a88da.mid,"[electronic, pop]"
4,lmd_full/f/f5ac97af893d6b5cec0324bc49ae10bb.mid,"[electronic, experimental]"


In [ ]:
unique_genres = set()
for genres_list in df['genre']:
    for genre in genres_list:
        unique_genres.add(genre)

NUM_CLASSES = len(unique_genres)

print(f"Number of unique genres: {NUM_CLASSES}")
print(f"Unique genres: {unique_genres}")

Number of unique genres: 42
Unique genres: {'funk', 'instrumentalrock', 'easylistening', 'instrumentalpop', 'symphonic', 'chillout', 'punkrock', 'jazz', 'ambient', '90s', 'drumnbass', 'metal', 'hiphop', 'swing', 'newage', 'alternative', 'blues', 'dance', 'house', 'folk', 'experimental', 'soundtrack', 'indie', '80s', 'orchestral', 'poprock', 'jazzfusion', 'techno', 'pop', 'popfolk', 'lounge', 'electropop', 'electronic', 'reggae', 'rock', 'classical', 'latin', 'synthpop', 'world', 'celtic', 'trance', 'country'}


In [ ]:
CLASS_TO_INDEX = {genre: idx for idx, genre in enumerate(sorted(list(unique_genres)))}

print("Genre to index mapping:")
print(CLASS_TO_INDEX)

# # Function to convert a list of genre names to a list of genre indices
# def get_genre_indices(genres):
#     return [CLASS_TO_INDEX[g] for g in genres]

# # Apply the function to create a new column 'genre_indices'
# df['genre_indices'] = df['genre'].apply(get_genre_indices)

# print("\nDataFrame with genre indices:")
# display(df.head())


Genre to index mapping:
{'80s': 0, '90s': 1, 'alternative': 2, 'ambient': 3, 'blues': 4, 'celtic': 5, 'chillout': 6, 'classical': 7, 'country': 8, 'dance': 9, 'drumnbass': 10, 'easylistening': 11, 'electronic': 12, 'electropop': 13, 'experimental': 14, 'folk': 15, 'funk': 16, 'hiphop': 17, 'house': 18, 'indie': 19, 'instrumentalpop': 20, 'instrumentalrock': 21, 'jazz': 22, 'jazzfusion': 23, 'latin': 24, 'lounge': 25, 'metal': 26, 'newage': 27, 'orchestral': 28, 'pop': 29, 'popfolk': 30, 'poprock': 31, 'punkrock': 32, 'reggae': 33, 'rock': 34, 'soundtrack': 35, 'swing': 36, 'symphonic': 37, 'synthpop': 38, 'techno': 39, 'trance': 40, 'world': 41}


In [ ]:
midi_fp = "/content/" + df['location'][0]
labels = df['genre'][0]
midi_fp, labels

('/content/lmd_full/2/2a604c93ed4ee9d82d388da694e89e7c.mid',
 array(['reggae', 'pop'], dtype=object))

In [ ]:
import miditoolkit


def load_midi(file_path=None, file=None, midi_checker='default'):
    """
    Open and check MIDI file, return MIDI object by miditoolkit.
    :param file_path:
    :param midi_checker:
    :return:
    """
    midi_obj = miditoolkit.midi.parser.MidiFile(filename=file_path, file=file)

    if midi_checker is not None and midi_checker != 'none':
        if isinstance(midi_checker, str):
            if midi_checker == 'default':
                midi_checker = default_check_midi
            else:
                raise ValueError("midi checker does not support value: %s" % midi_checker)

        midi_checker(midi_obj)

    return midi_obj


def default_check_midi(midi_obj):
    # check abnormal values in parse result
    max_time_length = 2 ** 31
    assert all(0 <= j.start < max_time_length
               and 0 <= j.end < max_time_length
               for i in midi_obj.instruments for j in i.notes), 'Bad note time'
    assert all(0 < j.numerator < max_time_length and 0 < j.denominator < max_time_length for j in
               midi_obj.time_signature_changes), 'Bad time signature value'
    assert 0 < midi_obj.ticks_per_beat < max_time_length, 'Bad ticks per beat'

    midi_notes_count = sum(len(inst.notes) for inst in midi_obj.instruments)
    assert midi_notes_count > 0, 'Blank note.'

In [ ]:
loaded_midi = load_midi(file_path=midi_fp)
print(loaded_midi)

ticks per beat: 96
max tick: 31873
tempo changes: 1
time sig: 1
key sig: 0
markers: 0
lyrics: False
instruments: 10


In [ ]:
mt = MultiTrack.from_midi(midi_fp)
mt

MultiTrack: 83 bars

In [ ]:
print(mt[0])
print(mt[1])

Bar 0: 0 insts
Bar 1: 5 insts


In [ ]:
# Convert to REMI-z format
import textwrap

clip = mt
print(type(clip))
remiz_format = clip.to_remiz_str(
    with_ts=True,
    with_tempo=True,
    with_velocity=False
    )
print(type(remiz_format))
print(f"Number of tokens in REMI-z string: {len(remiz_format.split())}")

wrapped_output = textwrap.fill(remiz_format, width=100)
print(wrapped_output)

<class 'remi_z.core.MultiTrack'>
<class 'str'>
Number of tokens in REMI-z string: 9623
s-9 t-33 b-1 s-9 t-33 i-58 o-0 p-73 d-25 o-24 p-77 d-8 o-30 p-78 d-7 o-36 p-80 d-6 o-42 p-73 d-6 i-0
o-6 p-68 d-3 p-65 d-3 p-61 d-3 o-18 p-68 d-3 p-65 d-3 p-61 d-2 o-30 p-68 d-3 p-65 d-3 p-61 d-2 o-42
p-68 d-3 p-65 d-2 p-61 d-2 i-35 o-6 p-68 d-3 p-65 d-3 p-61 d-3 o-18 p-68 d-3 p-65 d-3 p-61 d-2 o-30
p-68 d-3 p-65 d-3 p-61 d-2 o-42 p-68 d-3 p-65 d-2 p-61 d-2 i-39 o-0 p-37 d-9 o-21 p-32 d-2 o-24 p-37
d-6 b-1 s-9 t-33 i-58 o-0 p-78 d-22 o-24 p-89 d-3 o-27 p-85 d-2 o-30 p-89 d-4 o-33 p-85 d-2 o-36
p-89 d-3 o-39 p-85 d-2 o-42 p-89 d-4 o-45 p-85 d-2 i-0 o-6 p-70 d-2 p-66 d-3 p-61 d-2 o-18 p-70 d-2
p-66 d-3 p-61 d-2 o-30 p-70 d-2 p-66 d-2 p-61 d-2 o-42 p-70 d-2 p-66 d-2 p-61 d-2 i-35 o-6 p-70 d-2
p-66 d-3 p-61 d-2 o-18 p-70 d-2 p-66 d-3 p-61 d-2 o-30 p-70 d-2 p-66 d-2 p-61 d-2 o-42 p-70 d-2 p-66
d-2 p-61 d-2 i-39 o-0 p-42 d-6 o-21 p-37 d-2 o-24 p-42 d-6 o-36 p-42 d-9 b-1 s-9 t-33 i-58 o-0 p-73
d-25 o-24 p-7

In [ ]:
mt = MultiTrack.from_midi(midi_fp)

In [ ]:
from remi_z import MultiTrack

def filter_empty_bars(multi_track_obj):
    non_empty_bars = []

    # Iterate through each bar of the input MultiTrack object
    for bar_index in range(len(multi_track_obj)):
        bar = multi_track_obj[bar_index]

        # Check if the current bar contains any instruments
        if len(bar) > 0:
            non_empty_bars.append(bar)

    # Create a new MultiTrack object from the non_empty_bars list
    new_multi_track = MultiTrack(non_empty_bars)

    return new_multi_track

print(f"Original MultiTrack: {len(mt)} bars")
filtered_mt = filter_empty_bars(mt)
print(f"Filtered MultiTrack (non-empty bars): {len(filtered_mt)} bars")

Original MultiTrack: 83 bars
Filtered MultiTrack (non-empty bars): 82 bars


In [ ]:
def segment_into_n_bars(multi_track_obj, n):
    segmented_tracks = []
    num_bars = len(multi_track_obj)

    for i in range(0, num_bars, n):
        # Extract a segment of bars. Access the .bars attribute to get the list of Bar objects.
        segment_multi_track = multi_track_obj[i : i + n]
        segment_bars = segment_multi_track.bars

        # Create a new MultiTrack object for this segment
        segmented_track = MultiTrack(segment_bars)
        segmented_tracks.append(segmented_track)

    return segmented_tracks


print(f"Number of bars in filtered MultiTrack: {len(filtered_mt)}")
segmented_list = segment_into_n_bars(filtered_mt, 8)
print(f"Number of 8-bar segments created: {len(segmented_list)}")

# Print the number of bars in each segment to verify
for i, segment in enumerate(segmented_list):
    print(f"Segment {i+1} has {len(segment)} bars")

Number of bars in filtered MultiTrack: 82
Number of 8-bar segments created: 11
Segment 1 has 8 bars
Segment 2 has 8 bars
Segment 3 has 8 bars
Segment 4 has 8 bars
Segment 5 has 8 bars
Segment 6 has 8 bars
Segment 7 has 8 bars
Segment 8 has 8 bars
Segment 9 has 8 bars
Segment 10 has 8 bars
Segment 11 has 2 bars


In [ ]:
import textwrap

# Convert the first 8-bar segment to its REMI-z string representation
first_segment = segmented_list[0]
first_segment_remiz_str = first_segment.to_remiz_str(
    with_ts=True,
    with_tempo=True,
    with_velocity=False
)

print("\nREMI-z string representation of the first 8-bar segment:")
print(f"Number of tokens in the first segment's REMI-z string: {len(first_segment_remiz_str.split())}")
wrapped_output_first_segment = textwrap.fill(first_segment_remiz_str, width=100)
print(wrapped_output_first_segment)


REMI-z string representation of the first 8-bar segment:
Number of tokens in the first segment's REMI-z string: 779
s-9 t-33 i-58 o-0 p-73 d-25 o-24 p-77 d-8 o-30 p-78 d-7 o-36 p-80 d-6 o-42 p-73 d-6 i-0 o-6 p-68 d-3
p-65 d-3 p-61 d-3 o-18 p-68 d-3 p-65 d-3 p-61 d-2 o-30 p-68 d-3 p-65 d-3 p-61 d-2 o-42 p-68 d-3 p-65
d-2 p-61 d-2 i-35 o-6 p-68 d-3 p-65 d-3 p-61 d-3 o-18 p-68 d-3 p-65 d-3 p-61 d-2 o-30 p-68 d-3 p-65
d-3 p-61 d-2 o-42 p-68 d-3 p-65 d-2 p-61 d-2 i-39 o-0 p-37 d-9 o-21 p-32 d-2 o-24 p-37 d-6 b-1 s-9
t-33 i-58 o-0 p-78 d-22 o-24 p-89 d-3 o-27 p-85 d-2 o-30 p-89 d-4 o-33 p-85 d-2 o-36 p-89 d-3 o-39
p-85 d-2 o-42 p-89 d-4 o-45 p-85 d-2 i-0 o-6 p-70 d-2 p-66 d-3 p-61 d-2 o-18 p-70 d-2 p-66 d-3 p-61
d-2 o-30 p-70 d-2 p-66 d-2 p-61 d-2 o-42 p-70 d-2 p-66 d-2 p-61 d-2 i-35 o-6 p-70 d-2 p-66 d-3 p-61
d-2 o-18 p-70 d-2 p-66 d-3 p-61 d-2 o-30 p-70 d-2 p-66 d-2 p-61 d-2 o-42 p-70 d-2 p-66 d-2 p-61 d-2
i-39 o-0 p-42 d-6 o-21 p-37 d-2 o-24 p-42 d-6 o-36 p-42 d-9 b-1 s-9 t-33 i-58 o-0 p

In [ ]:
# def create_classification_samples(midi_fp, labels):

#     label_vector = [0] * NUM_CLASSES
#     for genre in labels:
#         if genre in GENRE_TO_INDEX:
#             label_vector[GENRE_TO_INDEX[genre]] = 1

#     mt = MultiTrack.from_midi(midi_fp)
#     filtered_mt = filter_empty_bars(mt)
#     bar_segments = segment_into_n_bars(filtered_mt, 8)

#     final_dataset = []
#     for segment in bar_segments:
#         segment_remiz_str = segment.to_remiz_str(
#             with_ts=True,
#             with_tempo=True,
#             with_velocity=False
#         )
#         final_dataset.append({
#             "input": segment_remiz_str,
#             "labels": label_vector
#             })

#     return final_dataset

# create_classification_samples(midi_fp, labels)

In [ ]:
# from tqdm import tqdm

# classification_samples = []

# for idx, row in tqdm(df.iterrows(), total=len(df), desc="Genrating Classification samples"):
#     midi_fp = "/content/" + row['location']
#     labels = row['genre']

#     samples = create_classification_samples(midi_fp, labels)
#     classification_samples.extend(samples)


In [ ]:
def midi_to_remiz_str(midi_fp):
    midi_fp = "/content/" + midi_fp
    mt = MultiTrack.from_midi(midi_fp)
    remiz_str = mt.to_remiz_str(
        with_ts=True,
        with_tempo=True,
        with_velocity=False
    )
    return remiz_str

midi_to_remiz_str(df['location'][0])

's-9 t-33 b-1 s-9 t-33 i-58 o-0 p-73 d-25 o-24 p-77 d-8 o-30 p-78 d-7 o-36 p-80 d-6 o-42 p-73 d-6 i-0 o-6 p-68 d-3 p-65 d-3 p-61 d-3 o-18 p-68 d-3 p-65 d-3 p-61 d-2 o-30 p-68 d-3 p-65 d-3 p-61 d-2 o-42 p-68 d-3 p-65 d-2 p-61 d-2 i-35 o-6 p-68 d-3 p-65 d-3 p-61 d-3 o-18 p-68 d-3 p-65 d-3 p-61 d-2 o-30 p-68 d-3 p-65 d-3 p-61 d-2 o-42 p-68 d-3 p-65 d-2 p-61 d-2 i-39 o-0 p-37 d-9 o-21 p-32 d-2 o-24 p-37 d-6 b-1 s-9 t-33 i-58 o-0 p-78 d-22 o-24 p-89 d-3 o-27 p-85 d-2 o-30 p-89 d-4 o-33 p-85 d-2 o-36 p-89 d-3 o-39 p-85 d-2 o-42 p-89 d-4 o-45 p-85 d-2 i-0 o-6 p-70 d-2 p-66 d-3 p-61 d-2 o-18 p-70 d-2 p-66 d-3 p-61 d-2 o-30 p-70 d-2 p-66 d-2 p-61 d-2 o-42 p-70 d-2 p-66 d-2 p-61 d-2 i-35 o-6 p-70 d-2 p-66 d-3 p-61 d-2 o-18 p-70 d-2 p-66 d-3 p-61 d-2 o-30 p-70 d-2 p-66 d-2 p-61 d-2 o-42 p-70 d-2 p-66 d-2 p-61 d-2 i-39 o-0 p-42 d-6 o-21 p-37 d-2 o-24 p-42 d-6 o-36 p-42 d-9 b-1 s-9 t-33 i-58 o-0 p-73 d-25 o-24 p-77 d-8 o-30 p-78 d-7 o-36 p-80 d-6 o-42 p-73 d-6 i-0 o-6 p-68 d-3 p-65 d-3 p-61 d-2 o-1

In [ ]:
# df['remiz_str'] = df['location'].progress_apply(midi_to_remiz_str)

In [ ]:
df.head()

,location,genre,remiz_str
0,lmd_full/2/2a604c93ed4ee9d82d388da694e89e7c.mid,"[reggae, pop]",s-9 t-33 b-1 s-9 t-33 i-58 o-0 p-73 d-25 o-24 ...
1,lmd_full/0/0fd51dae1d5ffc871de6b8bd8ff3e5ce.mid,"[pop, popfolk]",s-9 t-33 b-1 s-9 t-33 i-28 o-30 p-48 d-3 o-36 ...
2,lmd_full/0/00bb4be2822c3828935498af7b90b98e.mid,"[pop, jazz]",s-9 t-25 b-1 s-9 t-25 i-0 o-2 p-61 d-7 p-59 d-...
3,lmd_full/e/e7394f75f6978369eb463abcb75a88da.mid,"[electronic, pop]",s-9 t-42 b-1 s-9 t-42 i-33 o-0 p-35 d-4 o-12 p...
4,lmd_full/f/f5ac97af893d6b5cec0324bc49ae10bb.mid,"[electronic, experimental]",s-9 t-38 i-81 o-6 p-53 d-6 p-48 d-6 o-30 p-53 ...


In [ ]:
# df.to_csv('midi_to_remiz_str.csv', index=False)

## Model Configuration

In [ ]:
# Load model
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("LongshenOu/m2m_pt")
model = AutoModelForCausalLM.from_pretrained("LongshenOu/m2m_pt")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/175M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Check the Special Tokens (BOS, EOS, PAD, UNK)
print("--- Special Tokens ---")
print(f"BOS Token (Start): {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")
print(f"EOS Token (End):   {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"PAD Token (Padding): {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"UNK Token (Unknown): {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")

--- Special Tokens ---
BOS Token (Start): [BOS] (ID: 2)
EOS Token (End):   [EOS] (ID: 1)
PAD Token (Padding): [EOS] (ID: 1)
UNK Token (Unknown): [UNK] (ID: 0)


In [ ]:
# Check the Vocabulary Size
print("\n--- Vocabulary Size ---")
print(f"Vocabulary Size: {len(tokenizer.vocab)}")
print(tokenizer.vocab)


--- Vocabulary Size ---
Vocabulary Size: 989
{'s-209': 895, 'i-34': 175, 'p-243': 513, 's-21': 707, 'o-96': 109, 'i-104': 245, 'o-2': 15, 'o-52': 65, 'p-66': 336, 'd-83': 609, 't-25': 965, 'o-56': 69, 'p-196': 466, 'v-13': 667, 't-21': 961, 'v-17': 671, 'd-51': 577, 'i-57': 198, 's-169': 855, 'd-96': 622, 'p-100': 370, 'd-76': 602, 'o-93': 106, 'p-79': 349, 'p-61': 331, 's-251': 937, 'p-129': 399, 'i-92': 233, 'p-162': 432, 's-26': 712, 'v-3': 657, 'p-72': 342, 'i-66': 207, 'd-4': 530, 's-110': 796, '[INST]': 9, 'p-244': 514, 's-190': 876, 'i-5': 146, 's-149': 835, 's-117': 803, 'p-215': 485, 'o-12': 25, 'p-12': 282, 'p-140': 410, 'o-45': 58, 's-225': 911, 'i-84': 225, 's-62': 748, 'p-21': 291, 's-81': 767, 'd-32': 558, 'd-117': 643, 'p-109': 379, 'i-85': 226, 'o-37': 50, 'p-173': 443, 's-207': 893, 's-42': 728, 'o-106': 119, 'v-4': 658, 's-230': 916, 'o-111': 124, 's-6': 692, 't-27': 967, 'i-110': 251, 'p-184': 454, 'd-14': 540, 'i-106': 247, 'p-154': 424, 'd-57': 583, 'p-144': 414, 

In [ ]:
# Get a Sample of the Vocabulary
print("\n--- Sample Vocabulary (First 20 tokens) ---")
# The tokenizer's vocab is a dictionary mapping token strings to IDs.
id_to_token = {v: k for k, v in tokenizer.vocab.items()}
for i in range(20):
    print(f"ID {i}: {id_to_token.get(i, '[Token not found]')}")


--- Sample Vocabulary (First 20 tokens) ---
ID 0: [UNK]
ID 1: [EOS]
ID 2: [BOS]
ID 3: [CLS]
ID 4: [SEP]
ID 5: [PAD]
ID 6: [MASK]
ID 7: [HIST]
ID 8: [PITCH]
ID 9: [INST]
ID 10: [MELODY]
ID 11: [CHORD]
ID 12: b-1
ID 13: o-0
ID 14: o-1
ID 15: o-2
ID 16: o-3
ID 17: o-4
ID 18: o-5
ID 19: o-6


In [ ]:
sample_remiz_input = "s-3 t-36 i-73 o-0 p-79 d-8 o-9 p-79 d-3 o-12 p-78 d-5 o-18 p-79 d-5 o-24 p-81 d-5 o-30 p-83 d-5"
tokenized_input = tokenizer(sample_remiz_input)
print(tokenized_input)

{'input_ids': [2, 689, 976, 214, 13, 349, 534, 22, 349, 529, 25, 348, 531, 31, 349, 531, 37, 351, 531, 43, 353, 531, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
print(tokenizer.convert_ids_to_tokens(tokenized_input['input_ids']))

['[BOS]', 's-3', 't-36', 'i-73', 'o-0', 'p-79', 'd-8', 'o-9', 'p-79', 'd-3', 'o-12', 'p-78', 'd-5', 'o-18', 'p-79', 'd-5', 'o-24', 'p-81', 'd-5', 'o-30', 'p-83', 'd-5', '[EOS]']


In [ ]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(989, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=989, bias=False)
)


In [ ]:
print("\n--- Model Configuration ---")
print(model.config)


--- Model Configuration ---
GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 2,
  "dtype": "float32",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.3",
  "use_cache": true,
  "vocab_size": 989
}



In [ ]:
print(f"Max sequence length of the model (n_positions): {model.config.n_positions}")

Max sequence length of the model (n_positions): 2048


In [ ]:
print(NUM_CLASSES)

42


In [ ]:
print(CLASS_TO_INDEX)

{'80s': 0, '90s': 1, 'alternative': 2, 'ambient': 3, 'blues': 4, 'celtic': 5, 'chillout': 6, 'classical': 7, 'country': 8, 'dance': 9, 'drumnbass': 10, 'easylistening': 11, 'electronic': 12, 'electropop': 13, 'experimental': 14, 'folk': 15, 'funk': 16, 'hiphop': 17, 'house': 18, 'indie': 19, 'instrumentalpop': 20, 'instrumentalrock': 21, 'jazz': 22, 'jazzfusion': 23, 'latin': 24, 'lounge': 25, 'metal': 26, 'newage': 27, 'orchestral': 28, 'pop': 29, 'popfolk': 30, 'poprock': 31, 'punkrock': 32, 'reggae': 33, 'rock': 34, 'soundtrack': 35, 'swing': 36, 'symphonic': 37, 'synthpop': 38, 'techno': 39, 'trance': 40, 'world': 41}


In [ ]:
# --- Configuration Constants ---
MODEL_NAME = 'gpt2'
MAX_CHUNK_LENGTH = 256 # Reduced for memory
EMBEDDING_DIM = 768
BATCH_SIZE = 4
# NUM_CLASSES = 100

In [ ]:
import torch

# Sample input sequence
input_text = "s-3 t-36 i-73 o-0 p-79 d-8"

# Tokenize the input text, also returning the attention mask
encoded_inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=MAX_CHUNK_LENGTH,
    return_token_type_ids=False,
    padding="max_length",
    return_tensors="pt",
    return_attention_mask=True
)
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]

print(f"Input Text: {input_text}")
print(f"Input IDs: {input_ids.squeeze()[:20]}")
print(f"Attention Mask: {attention_mask.squeeze()[:20]}")
print(f"Input IDs Shape: {input_ids.shape}")
print(f"Attention Mask Shape: {attention_mask.shape}")



Input Text: s-3 t-36 i-73 o-0 p-79 d-8
Input IDs: tensor([  2, 689, 976, 214,  13, 349, 534,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Input IDs Shape: torch.Size([1, 512])
Attention Mask Shape: torch.Size([1, 512])


In [ ]:
print(tokenizer.convert_ids_to_tokens(input_ids.squeeze())[:20])

['[BOS]', 's-3', 't-36', 'i-73', 'o-0', 'p-79', 'd-8', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]', '[EOS]']


In [ ]:
# Generate a sequence using the model
output_sequences = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,

    max_new_tokens=50,  # Max length of the generated sequence
    num_return_sequences=1, # Number of sequences to generate
    no_repeat_ngram_size=2, # Avoid repeating n-grams
    do_sample=True, # Use sampling instead of greedy decoding
    top_k=50, # Consider only the top_k most likely tokens
    top_p=0.95, # Nucleus sampling
    temperature=0.7, # Control randomness
    pad_token_id=tokenizer.eos_token_id # Use EOS token for padding
)

# Decode the generated sequence
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

print(f"\nGenerated Output: {generated_text}")


Generated Output: s-3 t-36 i-73 o-0 p-79 d-8 p-45 d-12 o-8 p-81 d-1 o-9 p-79 d-1 o-10 p-81 d-2 o-12 p-79 d-2 o-14 p-81 d-4 o-18 p-79 d-6 p-45 d-6 o-24 p-77 d-6 p-47 d-12 o-30 p-76 d-6 o-36 p-77 d-2 o-38 p-79 d-4 o-42 p-77 d-3 p-47 d-6 o-45 p-76 d-3 b-1 s-3 t-35 i-73


In [ ]:
import collections

# Get the tokenizer's vocabulary
vocabulary = tokenizer.vocab

# Sort the vocabulary by value (ID) and create an OrderedDict
sorted_vocab_items = collections.OrderedDict(sorted(vocabulary.items(), key=lambda item: item[1]))

print("--- Sorted Vocabulary (First 20 entries by ID) ---")
# Iterate and print the first 20 items from the OrderedDict
for i, (token, token_id) in enumerate(sorted_vocab_items.items()):
    if i >= 20:
        break
    print(f"ID: {token_id}, Token: {token}")

--- Sorted Vocabulary (First 20 entries by ID) ---
ID: 0, Token: [UNK]
ID: 1, Token: [EOS]
ID: 2, Token: [BOS]
ID: 3, Token: [CLS]
ID: 4, Token: [SEP]
ID: 5, Token: [PAD]
ID: 6, Token: [MASK]
ID: 7, Token: [HIST]
ID: 8, Token: [PITCH]
ID: 9, Token: [INST]
ID: 10, Token: [MELODY]
ID: 11, Token: [CHORD]
ID: 12, Token: b-1
ID: 13, Token: o-0
ID: 14, Token: o-1
ID: 15, Token: o-2
ID: 16, Token: o-3
ID: 17, Token: o-4
ID: 18, Token: o-5
ID: 19, Token: o-6


In [ ]:
import json

output_file = "vocab.json"

with open(output_file, 'w') as f:
    json.dump(sorted_vocab_items, f, indent=4)

print(f"Tokenizer vocabulary successfully saved to {output_file}")

Tokenizer vocabulary successfully saved to vocab.json


In [ ]:
!ls

lmd_full  lmd_full.tar.gz  lmd_full.tar.gz.1  REMI-z  sample_data  vocab.json


# Finetuning
Fine-tune the pre-trained `GPT2LMHeadModel` from `LongshenOu/m2m_pt` for a classification task by extracting the base model as a feature extractor, defining a custom PyTorch classifier head, integrating the classifier with the feature extractor, and setting up a training loop with an optimizer and loss function.

## Load Base Model for Feature Extraction

Extract the base GPT2Model from the loaded GPT2LMHeadModel. This will remove the language modeling head, allowing us to use the model's hidden states as features.


In [ ]:
feature_extractor = model.transformer
print(f"Feature extractor model type: {type(feature_extractor)}")
print(feature_extractor)

Feature extractor model type: <class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
GPT2Model(
  (wte): Embedding(989, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)


### Freeze the parameters of `feature_extractor`

In [ ]:
for param in feature_extractor.parameters():
    param.requires_grad = False

In [ ]:
print("\n--- Checking feature_extractor parameters' requires_grad status ---")
for name, param in feature_extractor.named_parameters():
    if 'weight' in name or 'bias' in name:
        print(f"{name}: requires_grad = {param.requires_grad}")
        # Print only a few to avoid too much output
        if 'h.0.attn.c_attn.weight' in name:
            break



--- Checking feature_extractor parameters' requires_grad status ---
wte.weight: requires_grad = False
wpe.weight: requires_grad = False
h.0.ln_1.weight: requires_grad = False
h.0.ln_1.bias: requires_grad = False
h.0.attn.c_attn.weight: requires_grad = False


## Define Custom Classifier Head



In [ ]:
import torch.nn as nn


class ChunkAggregationHead(nn.Module):
    """
    Handles aggregation of multiple chunk embeddings into a single document embedding
    and performs the final multi-label classification.
    """
    def __init__(self, hidden_size, num_labels, dropout_prob=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout_prob)

        # Hierarchical Attention Mechanism (to weight the chunks)
        self.chunk_attention = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1),
            nn.Softmax(dim=1)  # Softmax over the 'num_chunks' dimension
        )

        # Final Multi-Label Classification Layer
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, chunk_embeddings):
        # chunk_embeddings shape: (batch_size, num_chunks, hidden_size)

        # 1. Apply Attention Mechanism
        # Weights shape: (batch_size, num_chunks, 1)
        attention_weights = self.chunk_attention(chunk_embeddings)

        # 2. Weighted Sum (Aggregation)
        # weighted_embeddings shape: (batch_size, num_chunks, hidden_size)
        weighted_embeddings = chunk_embeddings * attention_weights

        # document_embedding shape: (batch_size, hidden_size)
        document_embedding = torch.sum(weighted_embeddings, dim=1)

        # 3. Final Classification
        pooled_output = self.dropout(document_embedding)
        logits = self.classifier(pooled_output)

        return logits


# Instantiate the classification head
classifier_head = ChunkAggregationHead(EMBEDDING_DIM, NUM_CLASSES)

print(f"Instantiated Classification Head:\n{classifier_head}")

Instantiated Classification Head:
ChunkAggregationHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (chunk_attention): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): Tanh()
    (2): Linear(in_features=768, out_features=1, bias=True)
    (3): Softmax(dim=1)
  )
  (classifier): Linear(in_features=768, out_features=42, bias=True)
)


### Integrate Classifier with Feature Extractor

Combine the feature extractor (base model) and the custom classifier head into a single model for fine-tuning. This setup will allow gradients to flow back to the classifier head, training it on the extracted features.


In [ ]:
class GPT2ForHierarchicalClassification(nn.Module):
    def __init__(self, feature_extractor, aggregation_head):
        super().__init__()
        self.feature_extractor = feature_extractor
        self.aggregation_head = aggregation_head

        # ----------------------------------------------------
        # 🔥 Freeze the feature extractor parameters here 🔥
        # ----------------------------------------------------
        for param in self.feature_extractor.parameters():
             param.requires_grad = False

        # Ensure the tokenizer's padding side is set to 'left' for GPT-2 if it's used
        # self.feature_extractor.config.tokenizer_padding_side = "left"

    def forward(self, input_ids, attention_mask=None):
        # 1. Reshape Input: Flatten chunks and batch
        # input_ids shape: (batch_size, num_chunks, sequence_length)
        batch_size, num_chunks, sequence_length = input_ids.shape

        flat_input_ids = input_ids.view(-1, sequence_length)             # (B * N, L)
        flat_attention_mask = attention_mask.view(-1, sequence_length)   # (B * N, L)

        # 2. Pass flat inputs through the feature extractor (Frozen GPT-2)
        # Gradient calculation is skipped for the feature extractor
        with torch.no_grad():
            outputs = self.feature_extractor(
                input_ids=flat_input_ids,
                attention_mask=flat_attention_mask
            )

        # 3. Extract Chunk Embeddings
        # use the embedding of the last token (index -1) as the chunk representation
        # flat_hidden_state shape: (B * N, hidden_size)
        flat_hidden_state = outputs.last_hidden_state[:, -1, :]

        # 4. Reshape Output: Restore batch and chunk dimensions
        # chunk_embeddings shape: (batch_size, num_chunks, hidden_size)
        chunk_embeddings = flat_hidden_state.view(batch_size, num_chunks, -1)

        # 5. Pass chunk embeddings to the Aggregation Head
        logits = self.aggregation_head(chunk_embeddings)

        return logits

# Instantiate the combined model
model = GPT2ForHierarchicalClassification(feature_extractor, classifier_head)

# # Freeze the feature extractor parameters
# for param in model.feature_extractor.parameters():
#     param.requires_grad = False

print(f"Instantiated Combined Model:\n{model}")

Instantiated Combined Model:
GPT2ForHierarchicalClassification(
  (feature_extractor): GPT2Model(
    (wte): Embedding(989, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (aggregation_head): ChunkAggregationHead(
    (dropout): Dropout(p=0.1,

In [ ]:
print("\n--- Checking combined_model parameters' requires_grad status ---")
for name, param in model.named_parameters():
    if 'feature_extractor' in name and 'weight' in name or 'bias' in name:
        print(f"{name}: requires_grad = {param.requires_grad}")
        # Print only a few to avoid too much output
        if 'h.0.attn.c_attn.weight' in name: # Example of a specific parameter
            break


--- Checking combined_model parameters' requires_grad status ---
feature_extractor.wte.weight: requires_grad = False
feature_extractor.wpe.weight: requires_grad = False
feature_extractor.h.0.ln_1.weight: requires_grad = False
feature_extractor.h.0.ln_1.bias: requires_grad = False
feature_extractor.h.0.attn.c_attn.weight: requires_grad = False


## Prepare Training Data

In [ ]:
print(NUM_CLASSES)

42


In [ ]:
BARS_PER_SEGMENT = 8

In [ ]:
df.head()

,location,genre
0,lmd_full/2/2a604c93ed4ee9d82d388da694e89e7c.mid,"[reggae, pop]"
1,lmd_full/0/0fd51dae1d5ffc871de6b8bd8ff3e5ce.mid,"[pop, popfolk]"
2,lmd_full/0/00bb4be2822c3828935498af7b90b98e.mid,"[pop, jazz]"
3,lmd_full/e/e7394f75f6978369eb463abcb75a88da.mid,"[electronic, pop]"
4,lmd_full/f/f5ac97af893d6b5cec0324bc49ae10bb.mid,"[electronic, experimental]"


In [ ]:
def process_midi_to_segments(midi_fp, bars_per_segment, tokenizer, max_token_length):
    """
    Loads a MIDI file, filters empty bars, segments into n-bar chunks,
    converts each chunk to a ReMiZ string, and tokenizes it.
    """
    # print(midi_fp)
    try:
        # 1. Load the MIDI
        mt = MultiTrack.from_midi(midi_fp)
    except Exception as e:
        print(f"Error loading MIDI {midi_fp}: {e}")
        return []

    # 2. Filter Empty Bars
    non_empty_bars = []
    for bar_index in range(len(mt)):
        bar = mt[bar_index]
        if len(bar) > 0: # Check if the bar contains any instruments/notes
            non_empty_bars.append(bar)
    filtered_mt = MultiTrack(non_empty_bars)

    # 3. Segment into N-Bar Chunks
    segmented_tracks = []
    num_bars = len(filtered_mt)
    for i in range(0, num_bars, bars_per_segment):
        # Extract the bar objects for the segment
        segment_multi_track = filtered_mt[i : i + bars_per_segment]
        segment_bars = segment_multi_track.bars

        # Create a new MultiTrack object for this segment
        segmented_track = MultiTrack(segment_bars)

        # 4. Convert segment to ReMiZ string
        remiz_str = segmented_track.to_remiz_str(
            with_ts=True, with_tempo=True, with_velocity=False
        )

        # 5. Tokenize the ReMiZ string
        tokens = tokenizer.encode(remiz_str, add_special_tokens=True, max_length=max_token_length, truncation=True)
        segmented_tracks.append(tokens)

    return segmented_tracks

In [ ]:
print(process_midi_to_segments(df['location'][0], BARS_PER_SEGMENT, tokenizer))

[[2, 695, 973, 199, 13, 343, 551, 37, 347, 534, 43, 348, 533, 49, 350, 532, 55, 343, 532, 141, 19, 338, 529, 335, 529, 331, 529, 31, 338, 529, 335, 529, 331, 528, 43, 338, 529, 335, 529, 331, 528, 55, 338, 529, 335, 528, 331, 528, 176, 19, 338, 529, 335, 529, 331, 529, 31, 338, 529, 335, 529, 331, 528, 43, 338, 529, 335, 529, 331, 528, 55, 338, 529, 335, 528, 331, 528, 180, 13, 307, 535, 34, 302, 528, 37, 307, 532, 12, 695, 973, 199, 13, 348, 548, 37, 359, 529, 40, 355, 528, 43, 359, 530, 46, 355, 528, 49, 359, 529, 52, 355, 528, 55, 359, 530, 58, 355, 528, 141, 19, 340, 528, 336, 529, 331, 528, 31, 340, 528, 336, 529, 331, 528, 43, 340, 528, 336, 528, 331, 528, 55, 340, 528, 336, 528, 331, 528, 176, 19, 340, 528, 336, 529, 331, 528, 31, 340, 528, 336, 529, 331, 528, 43, 340, 528, 336, 528, 331, 528, 55, 340, 528, 336, 528, 331, 528, 180, 13, 312, 532, 34, 307, 528, 37, 312, 532, 49, 312, 535, 12, 695, 973, 199, 13, 343, 551, 37, 347, 534, 43, 348, 533, 49, 350, 532, 55, 343, 532, 141,

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class MIDIMultiLabelDataset(Dataset):
    def __init__(self, df, tokenizer, bars_per_segment, class_to_idx, num_classes, max_token_length):
        self.df = df
        self.tokenizer = tokenizer
        self.bars_per_segment = bars_per_segment
        self.class_to_idx = class_to_idx
        self.num_classes = num_classes
        self.max_token_length = max_token_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        midi_fp = "/content/" + row['location']
        labels_list = row['genre']

        # --- 1. MIDI Processing to Tokenized Chunks ---
        # Get a list of tokenized segments (chunks)
        tokenized_segments = process_midi_to_segments(midi_fp, self.bars_per_segment, self.tokenizer, self.max_token_length)

        if not tokenized_segments:
            # Handle empty/corrupted MIDI files by returning a placeholder or raising an error.
            raise ValueError(f"No valid segments found for MIDI at {midi_fp}")

        # --- 2. Token Padding (Required for Collation) ---
        # Find the max token length (L) across all segments in this *one* document
        max_segment_len = max(len(seg) for seg in tokenized_segments)

        padded_segments = []
        for segment in tokenized_segments:
            # Pad each segment to the max length L found in this document
            padding_needed = max_segment_len - len(segment)
            padded_segment = segment + [self.tokenizer.pad_token_id] * padding_needed
            padded_segments.append(padded_segment)

        # 3. Create Multi-Hot Vector
        multi_hot_vector = np.zeros(self.num_classes, dtype=np.float32)
        for class_name in labels_list:
            if class_name in self.class_to_idx:
                class_index = self.class_to_idx[class_name]
                multi_hot_vector[class_index] = 1.0

        # Output Tensors (Shape: [N, L] where L is max_segment_len for THIS document)
        input_ids = torch.tensor(padded_segments, dtype=torch.long)
        attention_mask = (input_ids != self.tokenizer.pad_token_id).int()

        return {
            'input_ids': input_ids,          # Shape: [N, L] (N and L are variable)
            'attention_mask': attention_mask, # Shape: [N, L]
            'labels': torch.tensor(multi_hot_vector, dtype=torch.float)
        }

In [ ]:
def midi_custom_collate_fn(batch):
    """
    Pads the variable number of chunks (N) AND the variable chunk length (L)
    across all documents in a batch.
    """

    # 1. Determine Max Dimensions (N_max and L_max) in the current batch
    max_num_chunks = max(item['input_ids'].shape[0] for item in batch)
    max_chunk_length = max(item['input_ids'].shape[1] for item in batch)

    # Assume the pad token ID is consistent across all documents
    pad_token_id = batch[0]['input_ids'].unique()[-1].item()

    batch_input_ids = []
    batch_attention_mask = []
    batch_labels = []

    for item in batch:
        current_input_ids = item['input_ids']
        current_attention_mask = item['attention_mask']
        num_chunks = current_input_ids.shape[0]

        # A. Inner Padding (Padding L): Pad all current segments to L_max
        # Pad columns (token length)
        col_padding_needed = max_chunk_length - current_input_ids.shape[1]

        # Pad tensor for columns (for each row/chunk)
        col_pad_tensor = torch.full((num_chunks, col_padding_needed), pad_token_id, dtype=torch.long)
        l_padded_input_ids = torch.cat([current_input_ids, col_pad_tensor], dim=1)

        # Pad attention mask columns with 0s
        col_pad_mask = torch.zeros((num_chunks, col_padding_needed), dtype=torch.int)
        l_padded_attention_mask = torch.cat([current_attention_mask, col_pad_mask], dim=1)


        # B. Outer Padding (Padding N): Pad the number of rows/chunks to N_max
        row_padding_needed = max_num_chunks - num_chunks

        # Pad rows (chunks) with L_max length pad tokens
        row_pad_tensor = torch.full((row_padding_needed, max_chunk_length), pad_token_id, dtype=torch.long)
        final_input_ids = torch.cat([l_padded_input_ids, row_pad_tensor], dim=0)

        # Pad rows (chunks) of mask with 0s
        row_pad_mask = torch.zeros((row_padding_needed, max_chunk_length), dtype=torch.int)
        final_attention_mask = torch.cat([l_padded_attention_mask, row_pad_mask], dim=0)

        batch_input_ids.append(final_input_ids)
        batch_attention_mask.append(final_attention_mask)
        batch_labels.append(item['labels'])

    return {
        'input_ids': torch.stack(batch_input_ids),        # Shape: [B, N_max, L_max]
        'attention_mask': torch.stack(batch_attention_mask), # Shape: [B, N_max, L_max]
        'labels': torch.stack(batch_labels)                 # Shape: [B, num_class]
    }

In [ ]:
train_dataset = MIDIMultiLabelDataset(
    df,
    tokenizer=tokenizer,
    bars_per_segment=BARS_PER_SEGMENT,
    class_to_idx=CLASS_TO_INDEX,
    num_classes=NUM_CLASSES,
    max_token_length=MAX_CHUNK_LENGTH
)

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=midi_custom_collate_fn, # ESSENTIAL for handling variable chunks/lengths
    shuffle=True
)

In [ ]:
# --- Fetch and Check the FIRST Batch ---
first_batch = next(iter(train_dataloader))

# The batch contains Doc A (3 segments, max L=320) and Doc B (2 segments, max L=410)

print("## 🎵 First Batch Data Sample Check 🎵")
print("---")

# Shape verification
input_ids = first_batch['input_ids']
labels = first_batch['labels']

B, N_max, L_max = input_ids.shape
print(f"BATCH SHAPE (B, N_max, L_max): ({B}, {N_max}, {L_max})")

# Expected N_max: 3 (from Doc A's 3 segments) or 5 (from Doc C's 5 segments)
# Expected L_max: 410 (from Doc B's longest segment) or 320/210 etc.

# Let's inspect the first batch (Doc A and Doc B):
# Doc A: N=3, L_docA=320. Doc B: N=2, L_docB=410.
# The DataLoader must output N_max=3, L_max=410.

print(f"Max Chunks (N_max): {N_max} (All documents padded to this number of rows/chunks)")
print(f"Max Segment Length (L_max): {L_max} (All chunks padded to this token length)")

print("\n### 1. Input IDs (Tokens) - [B, N_max, L_max]")
print(f"Shape: {input_ids.shape}")
print("--- Doc 1 (MIDI A) Chunk 1 (Padded to 410) ---")
print(input_ids[0, 0, :350]) # Should show token IDs then padding

print("\n--- Doc 2 (MIDI B) Chunk 1 (Padded to 410) ---")
print(input_ids[1, 0, :350]) # Should show token IDs then padding

print("\n--- Doc 2 (MIDI B) PADDED CHUNK (Should be all padding) ---")
print(input_ids[1, 2, 0:5]) # This should be the padding chunk (all pad_token_id=1)

print("\n### 2. Labels (Multi-Hot Vector) - [B, num_class]")
print(f"Shape: {labels.shape}")

# Doc A labels: class1, class10 -> indices 0 and 9
print("--- Labels Doc 1 ---")
print(labels[0, 0:15])

# Doc B labels: class50, class51, class52, class99 -> indices 49, 50, 51, 98
print("--- Labels Doc 2 ---")
print(labels[1, 48:53])
print(labels[1, 98:100])

## 🎵 First Batch Data Sample Check 🎵
---
BATCH SHAPE (B, N_max, L_max): (4, 26, 256)
Max Chunks (N_max): 26 (All documents padded to this number of rows/chunks)
Max Segment Length (L_max): 256 (All chunks padded to this token length)

### 1. Input IDs (Tokens) - [B, N_max, L_max]
Shape: torch.Size([4, 26, 256])
--- Doc 1 (MIDI A) Chunk 1 (Padded to 410) ---
tensor([  2, 692, 970, 193,  13, 337, 538,  12, 695, 970, 193,  13, 337, 538,
         25, 336, 538,  37, 337, 538,  49, 337, 538, 325, 538,  12, 695, 970,
        193,  13, 339, 538, 325, 538,  25, 339, 538, 324, 538,  37, 341, 538,
        325, 538,  49, 337, 538, 325, 538,  12, 695, 970, 193,  13, 342, 538,
        327, 538,  25, 342, 538, 327, 538,  37, 341, 538, 329, 538,  49, 341,
        538, 325, 538,  12, 695, 970, 193,  13, 339, 538, 330, 538,  25, 339,
        538, 330, 538,  37, 337, 538, 329, 538,  49, 344, 538, 329, 538,  12,
        695, 970, 193,  13, 342, 538, 327, 538,  25, 339, 538, 327, 538,  37,
        341, 538

### Data Pipeline: Dataset, DataLoader, and `collate_fn` Explained

Handling MIDI data for a Transformer model presents unique challenges, primarily due to the variable length of musical pieces. Our data pipeline, utilizing `MIDIMultiLabelDataset`, `DataLoader`, and a custom `midi_custom_collate_fn`, is designed to efficiently manage this variability and prepare data for the hierarchical classification model.

#### 1. `MIDIMultiLabelDataset` (The Data Source for a Single MIDI)

This class serves as the initial layer for processing individual MIDI files. When the `DataLoader` requests an item (`__getitem__(idx)` is called), `MIDIMultiLabelDataset` performs the following steps:

*   **Load MIDI and Segment**: It takes a MIDI file path and passes it to the `process_midi_to_segments` function.
    *   `process_midi_to_segments` loads the MIDI file into a `MultiTrack` object using the REMI-z library.
    *   It then filters out any empty bars to ensure only musically relevant content is processed.
    *   The filtered `MultiTrack` is segmented into fixed `BARS_PER_SEGMENT` chunks (e.g., 8-bar segments). This is crucial because different MIDI files can have wildly different total lengths, but we need consistent input sizes for our model's feature extractor.
    *   Each musical chunk (segment) is converted into its REMI-z string representation (a sequence of tokens).
    *   These REMI-z token sequences are then tokenized using the pre-trained `tokenizer` and padded or truncated to a uniform `MAX_CHUNK_LENGTH` (e.g., 256 tokens). If a document originally has `N` chunks, this step produces `N` tokenized sequences, each of length `MAX_CHUNK_LENGTH`.
*   **Per-Document Data Structure**: For a given MIDI document, the `MIDIMultiLabelDataset` returns `N` tokenized and padded chunks. The key here is that while each *chunk* has a fixed token length (`MAX_CHUNK_LENGTH`), the *number of chunks* (`N`) can still vary from one MIDI document to another.
*   **Create Multi-Hot Labels**: The list of genre names associated with the MIDI file (e.g., `['reggae', 'pop']`) is converted into a multi-hot encoded vector. This vector has a size of `NUM_CLASSES` (e.g., 42), with `1.0` at the indices corresponding to the present genres and `0.0` otherwise. This is the ground truth for multi-label classification.

**Output of `MIDIMultiLabelDataset` (`__getitem__`)**: For each MIDI document, it yields a dictionary containing:
    *   `input_ids`: `torch.Tensor` of shape `[num_chunks, MAX_CHUNK_LENGTH]`
    *   `attention_mask`: `torch.Tensor` of shape `[num_chunks, MAX_CHUNK_LENGTH]`
    *   `labels`: `torch.Tensor` of shape `[NUM_CLASSES]`

#### 2. `DataLoader` (The Batch Assembler)

The `DataLoader`'s primary role is to collect individual data samples from the `MIDIMultiLabelDataset` and group them into mini-batches. However, because the `num_chunks` dimension returned by `MIDIMultiLabelDataset` is *variable* across different MIDI documents, a standard `DataLoader` cannot directly stack these tensors. This is precisely why a custom `collate_fn` is indispensable.

*   It iteratively fetches `BATCH_SIZE` individual data items (dictionaries) from the `MIDIMultiLabelDataset`.
*   Instead of attempting to combine them directly, it passes this list of `BATCH_SIZE` items to our custom `midi_custom_collate_fn`.

#### 3. `midi_custom_collate_fn` (The Batch Unifier)

This is the most crucial component for harmonizing the variable dimensions within a batch. Its function is to take the list of `BATCH_SIZE` data items (each with its own `num_chunks`) and uniformly pad them so that they can be stacked into a single, cohesive tensor for the model.

Here's how it ensures uniformity:

*   **Determine Max Dimensions within the Batch**: It first scans all `BATCH_SIZE` documents within the current batch to identify:
    *   `N_max`: The maximum number of chunks found in any single document within that batch.
    *   `L_max`: The maximum token length for any chunk across all chunks in all documents within that batch. (Given our `process_midi_to_segments` already pads individual chunks to `MAX_CHUNK_LENGTH`, `L_max` will typically be `MAX_CHUNK_LENGTH`).
*   **Inner Padding (Token Length `L_max`)**: For any chunk whose tokenized length is less than `L_max` (which would only happen if `MAX_CHUNK_LENGTH` was not a fixed limit in `process_midi_to_segments`, or if we didn't truncate), it pads the token sequence with `tokenizer.pad_token_id` and extends its `attention_mask` with zeros.
*   **Outer Padding (Number of Chunks `N_max`)**: This is the key step for documents with fewer than `N_max` chunks. For such documents, `midi_custom_collate_fn` adds extra 'padding chunks'. These padding chunks are entirely filled with `tokenizer.pad_token_id` for their `input_ids` and `0`s for their `attention_mask`. This ensures that every document in the batch effectively has `N_max` chunks.
*   **Stacking**: After both inner (token length) and outer (number of chunks) padding, all tensors within the batch now have consistent dimensions. The `input_ids`, `attention_mask`, and `labels` from each document are then stacked into single `torch.Tensor`s.

**Output of `midi_custom_collate_fn`**: A single dictionary ready for direct input to the model:
    *   `input_ids`: `torch.Tensor` of shape `[BATCH_SIZE, N_max, MAX_CHUNK_LENGTH]`
    *   `attention_mask`: `torch.Tensor` of shape `[BATCH_SIZE, N_max, MAX_CHUNK_LENGTH]`
    *   `labels`: `torch.Tensor` of shape `[BATCH_SIZE, NUM_CLASSES]`

This structured batch is then fed to the `GPT2ForHierarchicalClassification` model, which can seamlessly process the `N_max` chunks and their `MAX_CHUNK_LENGTH` token sequences, ultimately aggregating their features for genre classification.

In [ ]:
print(model)

# Calculate and print model size information
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_params = total_params - trainable_params

# Estimate memory footprint (assuming float32 parameters, 4 bytes per parameter)
param_size_mb = total_params * 4 / (1024**2)

print(f"\n--- Model Size Information ---")
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Non-Trainable Parameters (Frozen): {non_trainable_params:,}")
print(f"Estimated Model Size (MB): {param_size_mb:.2f}")

GPT2ForHierarchicalClassification(
  (feature_extractor): GPT2Model(
    (wte): Embedding(989, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (aggregation_head): ChunkAggregationHead(
    (dropout): Dropout(p=0.1, inplace=False)
    (chunk_at

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
print("--- Dataset Memory Usage (Pandas DataFrame) ---")
df.info(memory_usage='deep')

--- Dataset Memory Usage (Pandas DataFrame) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2842 entries, 0 to 2841
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   location  2842 non-null   object
 1   genre     2842 non-null   object
dtypes: object(2)
memory usage: 599.6 KB


### Explanation of Memory Usage

*   **DataFrame Memory (RAM)**: The `df.info(memory_usage='deep')` output provides the memory footprint of your pandas DataFrame, which holds the metadata and file paths for the entire dataset. This is the main memory usage for the raw dataset itself in your system's RAM.

*   **Dataset and DataLoader Memory (Batch-wise)**:
    *   The `MIDIMultiLabelDataset` itself, when instantiated, primarily holds references to the DataFrame, tokenizer, and configuration parameters. It *does not* load all MIDI files or their tokenized representations into RAM at once. Instead, when `__getitem__` is called (typically by the `DataLoader`), it processes *one* MIDI file at a time, segments it, tokenizes it, and returns the tensors for that single item.
    *   The `DataLoader` then groups these individual items into batches. The memory footprint relevant here is for a *single batch* of processed data that gets loaded into GPU memory (or CPU memory if `device` is set to 'cpu').

    As seen from the `first_batch` check (which displayed `BATCH_SHAPE (B, N_max, L_max): (4, 26, 256)`):
    *   `B` (batch size): `4` documents per batch.
    *   `N_max` (max chunks per document in that batch): `26` chunks.
    *   `L_max` (max token length per chunk in that batch): `256` tokens.

    These `input_ids` and `attention_mask` tensors for a single batch would be `BATCH_SIZE * N_max * L_max` integers. If each integer (`torch.long`) takes 8 bytes, then each batch's input tensors alone would be `4 * 26 * 256 * 8 = 212,992` bytes (or approximately `0.2 MB`). The labels (`[B, num_classes]`) are much smaller. This batch data is dynamically loaded into GPU memory (or CPU) during each training step.

*   **GPU Memory for Training**: When training the model, the GPU needs to store several components:
    1.  **Model Parameters**: As previously calculated, your model's parameters alone take up approximately **335.74 MB** (for float32 weights).
    2.  **Optimizer States**: For optimizers like AdamW, internal states (e.g., momentum, variance) are maintained for each trainable parameter. This typically consumes about **2x the memory of the trainable parameters**. Since you have ~623K trainable parameters, this adds roughly `623,659 * 2 * 4 bytes/param / (1024^2) = ~4.75 MB`.
    3.  **Activations**: This is the most variable part. During the forward pass, intermediate activations are generated and stored for the backward pass (gradient computation). The memory consumed by activations is highly dependent on the `BATCH_SIZE`, `N_max` (number of chunks), and `L_max` (sequence length) of your input. Freezing the feature extractor helps here as activations for its layers don't need to be stored for gradient computation.

Therefore, the total GPU memory needed during training will be the sum of these, predominantly driven by the model parameters and the activations generated for the given batch size and sequence lengths. You could monitor `torch.cuda.max_memory_allocated()` or `nvidia-smi` during training for real-time usage.

In [ ]:
model.to(device)

GPT2ForHierarchicalClassification(
  (feature_extractor): GPT2Model(
    (wte): Embedding(989, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (aggregation_head): ChunkAggregationHead(
    (dropout): Dropout(p=0.1, inplace=False)
    (chunk_at

In [ ]:
import torch.optim as optim

# Define Optimizer (only for UN-frozen parameters)
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.AdamW(trainable_params, lr=1e-3)

# Define Loss Function (Binary Cross-Entropy for Multi-Label)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# from tqdm.auto import tqdm
# import torch.nn as nn

# NUM_EPOCHS = 1

# model.train()
# for epoch in range(NUM_EPOCHS):
#     total_loss = 0

#     for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
#         optimizer.zero_grad()

#         # Move data to the device
#         input_ids = batch['input_ids'].to(device)         # [B, N_max, L_max]
#         attention_mask = batch['attention_mask'].to(device) # [B, N_max, L_max]
#         labels = batch['labels'].to(device)               # [B, 100]

#         # 1. Forward Pass
#         # The model handles the flattening, feature extraction, and aggregation.
#         logits = model(input_ids, attention_mask)

#         # 2. Calculate Loss
#         loss = criterion(logits, labels)

#         # 3. Backward Pass and Optimization
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     avg_loss = total_loss / len(train_dataloader)
#     print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")

## Lightning Module

In [ ]:
# !pip install lightning # Install PyTorch Lightning

In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
import lightning as L

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 39.2 MB/s eta 0:00:00


In [ ]:
class GenreClassifier(L.LightningModule):
    def __init__(self, model_with_head):
        super().__init__()
        self.model_with_head = model_with_head
        # Use BCEWithLogitsLoss for multi-label classification
        self.criterion = torch.nn.BCEWithLogitsLoss()

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        logits = self.model_with_head(input_ids=input_ids, attention_mask=attention_mask)
        loss = self.criterion(logits, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        logits = self.model_with_head(input_ids=input_ids, attention_mask=attention_mask)
        loss = self.criterion(logits, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        # Optional: calculate accuracy or other metrics
        preds = (torch.sigmoid(logits) > 0.5).float()
        accuracy = (preds == labels).float().mean()
        self.log("val_acc", accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        logits = self.model_with_head(input_ids=input_ids, attention_mask=attention_mask)
        loss = self.criterion(logits, labels)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        preds = (torch.sigmoid(logits) > 0.5).float()
        accuracy = (preds == labels).float().mean()
        self.log("test_acc", accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        # Only optimize the parameters of the classifier head
        # Ensure the learning rate is appropriate
        return torch.optim.AdamW(self.model_with_head.aggregation_head.parameters(), lr=1e-4)

In [ ]:
import lightning as L
from torch.utils.data import random_split

class GenreClassificationData(L.LightningDataModule):
    def __init__(self, df, tokenizer, bars_per_segment, class_to_idx, num_classes, max_token_length, batch_size=BATCH_SIZE, val_split=0.1, test_split=0.1):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer
        self.bars_per_segment = bars_per_segment
        self.class_to_idx = class_to_idx
        self.num_classes = num_classes
        self.max_token_length = max_token_length
        self.batch_size = batch_size
        self.val_split = val_split
        self.test_split = test_split

    def setup(self, stage=None):
        # Assign train/val/test datasets for use in dataloaders
        full_dataset = MIDIMultiLabelDataset(self.df, self.tokenizer, self.bars_per_segment, self.class_to_idx, self.num_classes, self.max_token_length)

        val_size = int(len(full_dataset) * self.val_split)
        test_size = int(len(full_dataset) * self.test_split)
        train_size = len(full_dataset) - val_size - test_size

        self.train_dataset, self.val_dataset, self.test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, collate_fn=midi_custom_collate_fn)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=midi_custom_collate_fn)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=midi_custom_collate_fn)

# Instantiate the DataModule
genre_classification_data = GenreClassificationData(
    df=df, # Use the main dataframe
    tokenizer=tokenizer,
    bars_per_segment=BARS_PER_SEGMENT,
    class_to_idx=CLASS_TO_INDEX,
    num_classes=NUM_CLASSES,
    max_token_length=MAX_CHUNK_LENGTH,
    batch_size=BATCH_SIZE
)

print("GenreClassificationData instantiated. Call setup() to create datasets.")

GenreClassificationData instantiated. Call setup() to create datasets.


In [ ]:
# Call setup to create the datasets
genre_classification_data.setup()

print(f"Number of training samples: {len(genre_classification_data.train_dataset)}")
print(f"Number of validation samples: {len(genre_classification_data.val_dataset)}")
print(f"Number of test samples: {len(genre_classification_data.test_dataset)}")

# Example of getting a batch from the training dataloader
for i, batch in enumerate(genre_classification_data.train_dataloader()):
    print(f"Train Batch {i+1}:")
    print(f"  input_ids shape: {batch['input_ids'].shape}")
    print(f"  attention_mask shape: {batch['attention_mask'].shape}")
    print(f"  labels shape: {batch['labels'].shape}")
    if i == 0:
        break

# Instantiate the Lightning Module
lightning_model = GenreClassifier(model_with_head=model)

# Set up the Lightning Trainer
trainer = L.Trainer(max_epochs=NUM_EPOCHS, accelerator="gpu" if torch.cuda.is_available() else "cpu")

print("Starting Lightning training...")
trainer.fit(lightning_model, genre_classification_data)
print("Lightning training finished.")

Number of training samples: 2274
Number of validation samples: 284
Number of test samples: 284


INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Train Batch 1:
  input_ids shape: torch.Size([4, 17, 256])
  attention_mask shape: torch.Size([4, 17, 256])
  labels shape: torch.Size([4, 42])
Starting Lightning training...


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name            ┃ Type                              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model_with_head │ GPT2ForHierarchicalClassification │ 88.0 M │ train │     0 │
│ 1 │ criterion       │ BCEWithLogitsLoss                 │      0 │ train │     0 │
└───┴─────────────────┴───────────────────────────────────┴────────┴───────┴───────┘

Trainable params: 623 K                                                                                            
Non-trainable params: 87.4 M                                                                                       
Total params: 88.0 M                                                                                               
Total estimated model params size (MB): 352                                                                        
Modules in train mode: 172                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


Lightning training finished.


In [ ]:
import os

# Define the path to save the model
CHECKPOINT_DIR = "model_checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
model_save_path = os.path.join(CHECKPOINT_DIR, "genre_classifier_model.pth")

# Save the model's state dictionary
torch.save(lightning_model.model_with_head.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to model_checkpoints/genre_classifier_model.pth


In [ ]:
print("Starting model evaluation on the test set...")
trainer.test(lightning_model, genre_classification_data)
print("Model evaluation finished.")

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

Starting model evaluation on the test set...


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9621896743774414     │
│         test_loss         │    0.12024378776550293    │
└───────────────────────────┴───────────────────────────┘

Model evaluation finished.


In [ ]:
def predict_genre(midi_file_path, model, tokenizer, bars_per_segment, class_to_idx, num_classes, max_token_length, device, threshold=0.5, top_k=2):
    # Ensure model is in evaluation mode
    model.eval()

    # Prepare the MIDI data for inference
    try:
        tokenized_segments = process_midi_to_segments(midi_file_path, bars_per_segment, tokenizer, max_token_length)
    except ValueError as e:
        print(f"Error processing MIDI file: {e}")
        return []
    if not tokenized_segments:
        return []

    # Pad segments for this single document
    max_segment_len = max(len(seg) for seg in tokenized_segments)
    padded_segments = []
    for segment in tokenized_segments:
        padding_needed = max_segment_len - len(segment)
        padded_segment = segment + [tokenizer.pad_token_id] * padding_needed
        padded_segments.append(padded_segment)

    # Convert to tensors and add batch dimension (Batch size = 1 for single inference)
    input_ids = torch.tensor(padded_segments, dtype=torch.long).unsqueeze(0).to(device)
    attention_mask = (input_ids != tokenizer.pad_token_id).int().to(device)

    # Perform inference
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.sigmoid(logits).squeeze(0) # Remove batch dimension

    # Convert probabilities to genre labels
    predicted_genres = []
    idx_to_class = {v: k for k, v in class_to_idx.items()}

    for i, prob in enumerate(probabilities):
        if prob > threshold:
            predicted_genres.append((idx_to_class[i], prob.item()))

    # Sort by probability descending
    predicted_genres.sort(key=lambda x: x[1], reverse=True)

    # Return only the top_k genres
    return predicted_genres[:top_k]

# Example usage of inference
print("\n--- Performing Inference on a Sample MIDI ---")
# Use one of the MIDI files from your dataset for a quick test
sample_midi_path = "/content/" + df['location'][0]

# Get actual labels for the sample MIDI file
actual_labels = df[df['location'] == sample_midi_path.replace('/content/', '')]['genre'].iloc[0]

# Re-instantiate the model structure to load the saved weights
# Make sure the feature_extractor and classifier_head are defined as before
# For this example, 'model' is already our GPT2ForHierarchicalClassification instance
# Load the saved state dict
loaded_model = GPT2ForHierarchicalClassification(feature_extractor, classifier_head).to(device)
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.eval()

predicted_genres = predict_genre(
    midi_file_path=sample_midi_path,
    model=loaded_model,
    tokenizer=tokenizer,
    bars_per_segment=BARS_PER_SEGMENT,
    class_to_idx=CLASS_TO_INDEX,
    num_classes=NUM_CLASSES,
    max_token_length=MAX_CHUNK_LENGTH,
    device=device,
    threshold=0, # Adjust threshold as needed
    top_k=2 # Limit to top 2 predictions
)

print(f"MIDI File: {sample_midi_path}")
print(f"Actual Genres: {actual_labels}")

if predicted_genres:
    print("Predicted Genres:")
    for genre, prob in predicted_genres:
        print(f"  - {genre}: {prob:.4f}")
else:
    print(f"No genres predicted for {sample_midi_path} above the threshold.")


--- Performing Inference on a Sample MIDI ---
MIDI File: /content/lmd_full/2/2a604c93ed4ee9d82d388da694e89e7c.mid
Actual Genres: ['reggae' 'pop']
Predicted Genres:
  - electronic: 0.5856
  - pop: 0.1844
